In [2]:
import cv2
import sys
import numpy as np
import dlib
from imutils import face_utils
import imutils
detector=dlib.get_frontal_face_detector()
predictor=dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

def createBox(img,points,scale=5,masked=False,cropped=True):
    if masked:
        mask=np.zeros_like(img)
        mask=cv2.fillPoly(mask,[points],(255,255,255))
    
        img=cv2.bitwise_and(img,mask)
        #cv2.imshow('mask',img)
    if cropped:
        bbox=cv2.boundingRect(points)
        x,y,w,h=bbox
        imgcrop=img[y:y+h,x:x+w]
        imgcrop=cv2.resize(imgcrop,(0,0),None,scale,scale)
        return imgcrop
    else:
        return mask
img=cv2.imread('Trump.jpg')
img=cv2.resize(img,(0,0),None,0.5,0.5)
imgorg=img.copy()
gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
faces=detector(gray)
for face in faces:
    x1,y1=face.left(),face.top()
    x2,y2=face.right(),face.bottom()
    imgorg=cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),3)
    land=predictor(gray,face)
    
    
    mypoints=[]
    for n in range(68):
        x=land.part(n).x
        y=land.part(n).y
        land1=face_utils.shape_to_np(land)
        #mypoints=np.append(mypoints,land)
        mypoints.append([x,y])
        #cv2.circle(img,(x,y),5,(50,50,255),cv2.FILLED)
        #cv2.putText(img,str(n),(x,y-25),cv2.FONT_HERSHEY_COMPLEX_SMALL,0.7,(0,255,0),1)
        
        #np.concatenate
    mypoint=np.array(mypoints)
    imgleft=createBox(img,mypoint[48:61],3,masked=True,cropped=False)
    imgcolorlip=np.zeros_like(imgleft)
    imgcolorlip[:]=0,0,255
    imgcolorlip=cv2.bitwise_and(imgleft,imgcolorlip)
    imgcolorlip=cv2.GaussianBlur(imgcolorlip,(7,7),10)
    imgray=cv2.cvtColor(imgorg,cv2.COLOR_BGR2GRAY)
    imgray=cv2.cvtColor(imgray,cv2.COLOR_GRAY2BGR)
    imgcolorlip=cv2.addWeighted(imgray,1,imgcolorlip,0.4,0)
    cv2.imshow('lipcolor',imgcolorlip)
    cv2.imshow('masked',imgleft)
    
    #print(mypoint)
cv2.imshow('img',imgorg)
cv2.waitKey()
cv2.destroyAllWindows()